# BEIR Benchmarking for Azure AI Search (Part1)

In this part1 notebook, we will walk through the process of downloading the BEIR dataset, indexing it in Azure Cognitive Search, and running a few queries against it.


## Preparation


### Configuration


In [ ]:
import os

In [ ]:
# Recreate Index if True
recreate_index = False

# BEIR datasets to download
dataset_name = "scifact"

### Environment variables


In [ ]:
# Load environment variabls from .env file
from dotenv import load_dotenv

load_dotenv()

### Download BEIR datasets


In [ ]:
from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.lexical import BM25Search as BM25

In [ ]:
url = (
    "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(
        dataset_name
    )
)

# path for downloaded files
out_dir = "./datasets"

data_path = util.download_and_unzip(url, out_dir)

In [ ]:
corpus, queries, qrels = GenericDataLoader(data_path).load(
    split="test"
)  # pull data from corpus and queries

## Azure AI Search


### Connect to Azure AI Search


In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient

service_name = "benchmark-ai-search"
index_name = dataset_name + "-vector"

admin_key = os.environ["SEARCH_ADMIN_KEY"]
endpoint = "https://{}.search.windows.net/".format(service_name)

admin_client = SearchIndexClient(
    endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(admin_key)
)

search_client = SearchClient(
    endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(admin_key)
)

### Create Index


In [1]:
from azure.search.documents.models import (
    VectorizedQuery,
)
from azure.search.documents.indexes.models import (
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    HnswAlgorithmConfiguration,
    HnswParameters,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    SemanticSearch,
    SemanticPrioritizedFields,
    SimpleField,
    SearchableField,
    CorsOptions,
)

In [ ]:
if recreate_index == True:
    try:
        admin_client.delete_index(index_name)
    except Exception as e:
        print(e)

#### Configure vector search


In [ ]:
# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE,
            ),
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            ),
        ),
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        ),
    ],
)

#### Configure semantic search


In [ ]:
# import SemanticConfiguration from azure ai search
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        # prioritized_keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="text")],
    ),
)

semantic_settings = SemanticSearch(configurations=[semantic_config])

#### Other configuration


In [ ]:
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []

### Define Index


In [ ]:
fields = [
    SimpleField(name="corpusId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="text", type=SearchFieldDataType.String),
    SearchField(
        name="titleVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",
    ),
    SearchField(
        name="textVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",
    ),
]

In [ ]:
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_settings,
    scoring_profiles=scoring_profiles,
    cors_options=cors_options,
)

print(index)

try:
    result = admin_client.create_index(index)
    print("Index", result.name, "created")
except Exception as e:
    print(e)

#### Embedding data using OpenAI model


In [ ]:
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-05-15",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

model = "text-embedding-ada-002-v2"


# Generate Document Embeddings using OpenAI Ada 002
# Read the text-sample.json
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text, model=model):
    return openai_client.embeddings.create(input=[text], model=model).data[0].embedding


# create documents for corpus
documents = []
for id in corpus:
    # print(id)
    documents.append(
        {
            "corpusId": id,
            "title": corpus[id]["title"],
            "text": corpus[id]["text"],
            "titleVector": generate_embeddings(corpus[id]["title"]),
            "textVector": generate_embeddings(corpus[id]["text"]),
        }
    )

#### Upload dataset into Index


In [ ]:
try:
    # Upload documents to the index per 100 documents
    print("documents size is", len(documents))
    if len(documents) > 100:
        for i in range(0, len(documents), 100):
            result = search_client.upload_documents(documents=documents[i : i + 100])
            print("Upload of new document succeeded: {}".format(result[0].succeeded))
    else:
        result = search_client.upload_documents(documents=documents)
        print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as e:
    print(e)

In [ ]:
with open("scifact_vector.json", "w") as f:
    for line in documents:
        f.write(f"{line}\n")

## Test


### Full text search (simple)


In [ ]:
query = "Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging."

results = search_client.search(
    search_text=query, include_total_count=True, top=50, query_type="simple"
)
for result in results:
    print(result["@search.score"])

### Semantic search


In [ ]:
query = "Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging."

results = search_client.search(
    search_text=query,
    include_total_count=True,
    top=50,
    query_type="semantic",
    semantic_configuration_name="my-semantic-config",
)
for result in results:
    # print(result["@search.score"])
    print(result["@search.reranker_score"])

### Vector search


#### Hnsw


In [ ]:
query = "Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging."
vector_query = VectorizedQuery(
    vector=generate_embeddings(query),
    k_nearest_neighbors=50,
    fields="titleVector, textVector",
)

results = search_client.search(
    search_text=None, vector_queries=[vector_query], include_total_count=True, top=50
)
for result in results:
    print(result["@search.score"])
    # print(result["@search.reranker_score"])

#### Exhausive KNN


In [ ]:
query = "Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging."
vector_query = VectorizedQuery(
    vector=generate_embeddings(query),
    k_nearest_neighbors=50,
    fields="titleVector, textVector",
    exhaustive=True,
)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    include_total_count=True,
    top=50,
)
for result in results:
    print(result["@search.score"])
    # print(result["@search.reranker_score"])

### Hybrid search


In [ ]:
query = "Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging."
vector_query = VectorizedQuery(
    vector=generate_embeddings(query),
    k_nearest_neighbors=50,
    fields="titleVector, textVector",
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    include_total_count=True,
    top=50,
    query_type="semantic",
    semantic_configuration_name="my-semantic-config",
)
for result in results:
    # print(result["@search.score"])
    print(result["@search.reranker_score"])